In [2]:
import pandas as pd
import numpy as np
import os
import tqdm
import shutil

In [11]:
dataset = 'emma' # onion or emma
results_dir = f'../results/' + dataset + '/performance/'
#cutoffs = [1, 5, 10, 20]
cutoffs = [5]
cutoff_date = '2025_03_13'
metrics = ['Recall', 'Precision', 'nDCG', 'MRR', 'HR', 'EFD', 'EPC', 'ARP', 'PopREO', 'PopRSP', 'ItemCoverage', 'UserCoverage']
#metrics = ['Recall', 'Precision', 'nDCG', 'MRR', 'HR']

In [12]:
def find_modality(string):
    keyword = "modalites="
    start = string.find(keyword)
    if start == -1:
        return None

    start += len(keyword)
    end = string.find("_", start)
    if end == -1:
        end = len(string)

    modality = string[start:end].strip("'")
    return modality

In [22]:
# read all files in result folder
files = os.listdir(results_dir)
df = pd.DataFrame()

for cutoff in cutoffs:
    df_cut = pd.DataFrame()
    result_files_cutoff = [f for f in files if f.startswith(f'rec_cutoff_{cutoff}_')]
    for f in result_files_cutoff:
        df_tmp = pd.read_csv(results_dir + f, sep='\t')
        tmp_metrics = [m for m in metrics if m in df_tmp.columns]
        df_tmp.rename(columns={m: f'{m}@{cutoff}' for m in tmp_metrics}, inplace=True)
        df_tmp['modality'] = find_modality(df_tmp['model'].values[0])
        df_tmp['path'] = df_tmp['model']
        df_tmp['model'] = df_tmp['model'].str.split('_').str[0]
        df_tmp['date'] = f[-23:-4]
        df_cut = pd.concat([df_cut, df_tmp], axis=0)

    # keep only newest model results
    df_cut = df_cut.sort_values('date').drop_duplicates(subset=['model', 'modality'], keep='last')

    # drop models results before cutoff date
    df_cut = df_cut[df_cut['date'] >= cutoff_date]

    # merge with previous cutoffs
    if not df.empty:
        df = pd.merge(df, df_cut, on=['model', 'date', 'modality'], how='inner')
        print(df.shape)
    else:
        df = df_cut

df.sort_values('nDCG@5', inplace=True)
# display columns with @10 only
df[['model', 'modality', 'date', 'path'] + [c for c in df.columns if '@' in c]]
#df[['model', 'modality', 'date'] + ['Recall@1', 'Recall@5', 'Recall@10', 'nDCG@1', 'nDCG@5', 'nDCG@10']]

,model,modality,date,path,Recall@5,Precision@5,nDCG@5,MRR@5,HR@5,EFD@5,EPC@5,ARP@5,PopREO@5,PopRSP@5,ItemCoverage@5,UserCoverage@5
0,Reranker,None,2025_03_25_10_47_45,Reranker_cosine_5_emotion_LightGCN,0.198933,0.073729,0.132113,0.141530,0.323888,0.328967,0.055494,423.790837,0.890942,0.851692,186.0,3776.0
24,LightGCN,None,2025_03_25_10_25_30,LightGCN_seed=123_e=200_bs=128_lr=0$0005_facto...,0.198933,0.073729,0.164259,0.204034,0.323888,0.395977,0.067496,423.790837,0.890942,0.851692,186.0,3776.0
6,LightGCNM,audio'-'textual'-'visual'-'emotion,2025_03_25_10_13_52,LightGCNM_seed=123_e=200_bs=128_lr=0$005_facto...,0.324974,0.121239,0.263663,0.312566,0.494174,0.632037,0.108354,614.739301,0.875507,0.973138,127.0,3776.0
28,FeatureItemKNN,None,2025_03_25_10_25_30,FeatureItemKNN_nn=50_sim=cosine_msf=0$2_bin=Fa...,0.326087,0.119915,0.263746,0.311825,0.499470,0.673988,0.111242,511.164672,0.688670,0.951999,173.0,3776.0
22,LightGCNM,None,2025_03_25_10_25_30,LightGCNM_seed=123_e=200_bs=128_lr=0$005_facto...,0.331103,0.122299,0.266636,0.312999,0.496557,0.655014,0.110823,563.026801,0.842431,0.967469,138.0,3776.0
81,ItemKNN,None,2025_03_25_10_25_30,ItemKNN_nn=100_sim=cosine_imp=standard_bin=Fal...,0.373725,0.137235,0.306628,0.362531,0.559322,0.757531,0.126911,613.229025,0.753984,0.975010,144.0,3776.0


In [13]:
# export to comma seperated file
df.to_csv(f'../results/{dataset}_performance.csv', index=False)

In [14]:
for cutoff in cutoffs:
    df_cut = df[['model'] + [c for c in df.columns if f'@{cutoff}' in c]]
    df_cut.to_csv(f'../results/{dataset}_performance_{cutoff}.csv', index=False)